In [1]:
import numpy as np
import pandas as pd

# Reading the dataset

In [2]:
df1 = pd.read_csv('2022.06.12.csv')
df2 = pd.read_csv('2022.06.13.csv')
df3 = pd.read_csv('2022.06.14.csv')
df_dataset = pd.concat([df1, df2, df3])
df_dataset.reset_index(drop=True, inplace=True)
df_dataset.head()

,avg_ipt,bytes_in,bytes_out,dest_ip,dest_port,entropy,num_pkts_out,num_pkts_in,proto,src_ip,src_port,time_end,time_start,total_entropy,label,duration
0,0.0,0,0,786,31306.0,0.0,1,0,6,786,47613.0,1655074782132491,1655074782132491,0.0,outlier,0.0
1,0.0,0,0,786,24298.0,0.0,1,0,6,786,47613.0,1655074782573321,1655074782573321,0.0,outlier,0.0
2,0.0,0,0,786,31789.0,0.0,1,0,6,786,47613.0,1655074783148517,1655074783148517,0.0,outlier,0.0
3,0.0,0,0,786,29409.0,0.0,1,0,6,786,47613.0,1655074783149855,1655074783149855,0.0,outlier,0.0
4,0.0,0,0,786,60965.0,0.0,1,0,6,786,47613.0,1655074783167254,1655074783167254,0.0,outlier,0.0


# Merging into 1 dataframe

In [3]:
df_dataset = pd.concat([df1, df2, df3])
df_dataset.reset_index(drop=True, inplace=True)
len(df_dataset.columns)

16

# Displaying the data

In [ ]:
import seaborn as sns

In [4]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068376 entries, 0 to 1068375
Data columns (total 16 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   avg_ipt        1068376 non-null  float64
 1   bytes_in       1068376 non-null  int64  
 2   bytes_out      1068376 non-null  int64  
 3   dest_ip        1068376 non-null  int64  
 4   dest_port      964168 non-null   float64
 5   entropy        1068376 non-null  float64
 6   num_pkts_out   1068376 non-null  int64  
 7   num_pkts_in    1068376 non-null  int64  
 8   proto          1068376 non-null  int64  
 9   src_ip         1068376 non-null  int64  
 10  src_port       964168 non-null   float64
 11  time_end       1068376 non-null  int64  
 12  time_start     1068376 non-null  int64  
 13  total_entropy  1068376 non-null  float64
 14  label          1068376 non-null  object 
 15  duration       1068376 non-null  float64
dtypes: float64(6), int64(9), object(1)
memory usage: 130.4

In [5]:
df_dataset = df_dataset.drop(['num_pkts_out', 'num_pkts_in'], axis=1)

In [6]:
df_dataset.columns

Index(['avg_ipt', 'bytes_in', 'bytes_out', 'dest_ip', 'dest_port', 'entropy',
       'proto', 'src_ip', 'src_port', 'time_end', 'time_start',
       'total_entropy', 'label', 'duration'],
      dtype='object')

# Taking care of missing data, Removing duplicates

In [7]:
# replace +ve and -ve infinity with NaN
df_dataset.replace([np.inf, -np.inf], np.nan, inplace=True)

In [8]:
# drop missing values
df_dataset.dropna(inplace=True)

In [9]:
print(df_dataset.duplicated().sum())

4447


In [10]:
df_dataset.drop_duplicates(inplace = True)

In [11]:
print(df_dataset.duplicated().sum())

0


# Implementing binary classification

In [12]:
df_dataset["label"].value_counts()

label
benign       516220
outlier      365385
malicious     78116
Name: count, dtype: int64

In [13]:
old_value = 'outlier'
new_value = 0
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [14]:
old_value = 'malicious'
new_value = 1
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

In [15]:
old_value = 'benign'
new_value = 2
df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_34636\3502624556.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_dataset['label'] = df_dataset['label'].replace(old_value, new_value)


In [16]:
df_dataset["label"].value_counts()

label
2    516220
0    365385
1     78116
Name: count, dtype: int64

# Balancing the data (Undersampling)

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils import class_weight


In [18]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
# fig = go.Figure(data=[
#     go.Bar(name='Benign',
#            y=df_equal["label"].value_counts().values[[2]],
#            x=['Benign'],
#            text = df_equal["label"].value_counts()[[2]],
#            orientation='v',
#            textposition='outside',),
#     go.Bar(name='Malicious',
#            y=df_equal["label"].value_counts().values[[1]],
#            x=['Malicious'],
#            text = df_equal["label"].value_counts()[[1]],
#            orientation='v',
#            textposition='outside',),
#     go.Bar(name='outlier',
#            y=df_equal["label"].value_counts().values[[2]],
#            x=['outlier'],
#            text = df_equal["label"].value_counts()[[2]],
#            orientation='v',
#            textposition='outside',)
# ])
# # Change the bar mode
# fig.update_layout(
#                   width=800,
#                   height=600,
#                   title=f'Class Distribution',
#                   yaxis_title='Number of attacks',
#                   xaxis_title='Attack Name',)
# iplot(fig)

# Creating train/test split

In [19]:
df=df_dataset
train, test = train_test_split(df, test_size=0.2)

In [20]:
# Number of rows in train dataset
len(train)

767776

In [21]:
# rows in test dataset
len(test)

191945

In [22]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 191945 entries, 1031798 to 281934
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        191945 non-null  float64
 1   bytes_in       191945 non-null  int64  
 2   bytes_out      191945 non-null  int64  
 3   dest_ip        191945 non-null  int64  
 4   dest_port      191945 non-null  float64
 5   entropy        191945 non-null  float64
 6   proto          191945 non-null  int64  
 7   src_ip         191945 non-null  int64  
 8   src_port       191945 non-null  float64
 9   time_end       191945 non-null  int64  
 10  time_start     191945 non-null  int64  
 11  total_entropy  191945 non-null  float64
 12  label          191945 non-null  int64  
 13  duration       191945 non-null  float64
dtypes: float64(6), int64(8)
memory usage: 22.0 MB


In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 767776 entries, 223809 to 446868
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   avg_ipt        767776 non-null  float64
 1   bytes_in       767776 non-null  int64  
 2   bytes_out      767776 non-null  int64  
 3   dest_ip        767776 non-null  int64  
 4   dest_port      767776 non-null  float64
 5   entropy        767776 non-null  float64
 6   proto          767776 non-null  int64  
 7   src_ip         767776 non-null  int64  
 8   src_port       767776 non-null  float64
 9   time_end       767776 non-null  int64  
 10  time_start     767776 non-null  int64  
 11  total_entropy  767776 non-null  float64
 12  label          767776 non-null  int64  
 13  duration       767776 non-null  float64
dtypes: float64(6), int64(8)
memory usage: 87.9 MB


# Feature scaling

In [24]:
min_max_scaler = MinMaxScaler().fit(train[['avg_ipt', 'bytes_in', 'bytes_out', 'dest_port', 'entropy','src_port','total_entropy', 'duration']])

In [25]:
numerical_columns = ['avg_ipt', 'bytes_in', 'bytes_out', 'dest_port', 'entropy', 'src_port','total_entropy', 'duration']

In [26]:
train[numerical_columns] = min_max_scaler.transform(train[numerical_columns])

In [27]:
print(len(train))

767776


In [28]:
test[numerical_columns] = min_max_scaler.transform(test[numerical_columns])

# Counting number of records in each split

In [29]:
print("Full dataset:\n")
print("Benign: " + str(df["label"].value_counts()[[2]].sum()))
print("Malicious: " + str(df["label"].value_counts()[[1]].sum()))
print("outlier: " + str(df["label"].value_counts()[[0]].sum()))
print("---------------")

print("Training set:\n")
print("Benign: " + str(train["label"].value_counts()[[2]].sum()))
print("Malicious: " + str(train["label"].value_counts()[[1]].sum()))
print("outlier: " + str(train["label"].value_counts()[[0]].sum()))
print("---------------")

print("Test set:\n")
print("Benign: " + str(test["label"].value_counts()[[2]].sum()))
print("Malicious: " + str(test["label"].value_counts()[[1]].sum()))
print("outlier: " + str(test["label"].value_counts()[[0]].sum()))

Full dataset:

Benign: 516220
Malicious: 78116
outlier: 365385
---------------
Training set:

Benign: 413164
Malicious: 62609
outlier: 292003
---------------
Test set:

Benign: 103056
Malicious: 15507
outlier: 73382


# Creating X, y variables

In [30]:
y_train = np.array(train.pop('label'))# pop removes "Label" from the dataframe
X_train = train.values

In [31]:
y_test = np.array(test.pop("label")) # pop removes "Label" from the dataframe
X_test = test.values

In [32]:
# Checking unique values in the label column
df['label'].unique()

array([0, 2, 1], dtype=int64)

# Neural Network

In [33]:
import tensorflow as tf
print(tf.__version__)

2.18.0


In [34]:
from tensorflow import keras
print(keras.__version__)

3.8.0


In [35]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM,Conv1D,MaxPool1D,Dropout,Flatten,Reshape
from tensorflow.python.keras.losses import SparseCategoricalCrossentropy

In [36]:
model= Sequential()

LTSM

In [37]:
conv_layer=Conv1D(512, kernel_size=1, padding="same", activation='relu', input_shape=(X_train.shape[1], 1))
model.add(conv_layer)
print(type(conv_layer))  

<class 'keras.src.layers.convolutional.conv1d.Conv1D'>


d:\python\anac\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [38]:
model.add(MaxPool1D(pool_size=(2)))
model.add(LSTM(units=16,return_sequences=False,dropout=0.2))
model.add(Dense(units=3,activation='softmax'))

In [39]:
# defining loss function, optimizer, metrics and then compiling model
model.compile(loss='categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 13, 512)        │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 6, 512)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 16)             │        33,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 34,931 (136.45 KB)

 Trainable params: 34,931 (136.45 KB)

 Non-trainable params: 0 (0.00 B)

In [40]:
from tensorflow.keras.utils import to_categorical

num_classes = 3  # Set this to match your output layer
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)


In [41]:
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

y_train shape: (767776, 3)
y_test shape: (191945, 3)


In [42]:
history=model.fit(X_train, y_train, epochs=10, batch_size=512, validation_data=(X_test,y_test))

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 76s 48ms/step - accuracy: 0.5360 - loss: 0.9168 - val_accuracy: 0.5369 - val_loss: 0.9048
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 73s 49ms/step - accuracy: 0.5379 - loss: 0.9050 - val_accuracy: 0.5369 - val_loss: 0.9048
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 71s 47ms/step - accuracy: 0.5386 - loss: 0.9054 - val_accuracy: 0.5369 - val_loss: 0.9049
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 69s 46ms/step - accuracy: 0.5372 - loss: 0.9064 - val_accuracy: 0.5369 - val_loss: 0.9048
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 69s 46ms/step - accuracy: 0.5384 - loss: 0.9050 - val_accuracy: 0.5369 - val_loss: 0.9049
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 71s 47ms/step - accuracy: 0.5382 - loss: 0.9043 - val_accuracy: 0.5369 - val_loss: 0.9048
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 70s 47ms/step - accuracy: 0.5384 - loss: 0.9047 - val_accuracy: 0.5369 - val_loss: 0.9048
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 89s 51ms/step - accuracy: 0.5418 -

# Decision Tree Classifier

In [43]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier(criterion = 'entropy', max_depth=5)

parameters = {'max_depth': randint(2, 10), 'min_samples_split': randint(2, 10), 'min_samples_leaf': [1, 2, 3, 4, 5], 'criterion': ['gini', 'entropy']}

random_search = RandomizedSearchCV(dt, param_distributions=parameters, n_iter=20, cv=5, n_jobs=-1)
random_search.fit(X_train, y_train)
# best_accuracy = random_search.best_score_
# best_parameters = random_search.best_params_

# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

RandomizedSearchCV(cv=5,
                   estimator=DecisionTreeClassifier(criterion='entropy',
                                                    max_depth=5),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002D0E0E32660>,
                                        'min_samples_leaf': [1, 2, 3, 4, 5],
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002D0E46683B0>})

In [44]:
from sklearn import metrics
# dt = DecisionTreeClassifier(**random_search.best_params_)
dt.fit(X_train,y_train)

# Predicting Train & Test Results
y_dt = dt.predict(X_train)
y_pred_dt = dt.predict(X_test)

# Accuracy
print("Accuracy on Training set: ",metrics.accuracy_score(y_train, y_dt))
print("Accuracy on Testing set: ",metrics.accuracy_score(y_test, y_pred_dt))

Accuracy on Training set:  0.9598268244904764
Accuracy on Testing set:  0.9597801453541379


In [45]:
print(len(y_dt))
print(f"Malicious: {len(y_dt[y_dt==1])}")
print(f"Outliers: {len(y_dt[y_dt==0])}")
print(f"Benign: {len(y_dt[y_dt==2])}")

767776
Malicious: 767776
Outliers: 1535552
Benign: 0


# Random Forest Classifier

In [46]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=40)

parameters = {'n_estimators': randint(50, 100), 'max_depth': randint(2, 8), 'min_samples_split': randint(2, 10),
              'min_samples_leaf': randint(1, 10), 'max_features': ['sqrt', 'log2', None]}

# random_search = RandomizedSearchCV(rf, n_iter=20, cv=5, n_jobs=-1)
rf.fit(X_train, y_train)
# best_accuracy = rf.accuracy
# best_parameters = rf.best_params_

# print("Best Accuracy: {:.2f} %".format(best_accuracy*100))
# print("Best Parameters:", best_parameters)

RandomForestClassifier(n_estimators=40)

In [47]:
# best_rf = RandomForestClassifier(rf)
# best_rf.fit(X_train, y_train)

# Predicting Train & Test Results
y_rf = rf.predict(X_train)
y_pred_rf = rf.predict(X_test)

# Accuracy
print("Accuracy on Training set: ",metrics.accuracy_score(y_train, y_rf))
print("Accuracy on Testing set: ",metrics.accuracy_score(y_test, y_pred_rf))

Accuracy on Training set:  0.9999153398908015
Accuracy on Testing set:  0.9913308499830681


In [48]:
print(rf.predict(X_train[1].reshape(1,-1)))

[[0. 1. 0.]]


# XGBoost Gradient Boosting

In [49]:
# !pip install xgboost

In [50]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [51]:
xgb_train, xgb_test = xgb_model.score(X_train , y_train), xgb_model.score(X_test , y_test)

print(f"Training Score: {xgb_train}")
print(f"Test Score: {xgb_test}")

Training Score: 0.9901507731421665
Test Score: 0.9879861418635547


# SKLearn Gradient Boosting

In [52]:
y_train_skb = np.argmax(y_train, axis=1)  # Convert from one-hot to class labels
y_test_skb = np.argmax(y_test, axis=1)    # Do the same for y_test


print(y_train.shape)
print(np.unique(y_train)) 

(767776, 3)
[0. 1.]


In [58]:
y_train_skb.shape

(767776,)

In [53]:
from sklearn.ensemble import GradientBoostingClassifier

SKGB = GradientBoostingClassifier(random_state=42)
SKGB.fit(X_train, y_train_skb)
SKGB_train, SKGB_test = SKGB.score(X_train , y_train_skb), SKGB.score(X_test , y_test_skb)

print(f"Training Score: {SKGB_train}")
print(f"Test Score: {SKGB_test}")

Training Score: 0.9839536531488351
Test Score: 0.9839224777931178


# Light Gradient Boosting

In [54]:
# !pip install ligthgbm

In [59]:
from lightgbm import LGBMClassifier  # Correct import

# Initialize the model
lgb_model = LGBMClassifier(random_state=42)

# Train the model
lgb_model.fit(X_train, y_train_skb)

# Evaluate the model
lgb_train, lgb_test = lgb_model.score(X_train, y_train_skb), lgb_model.score(X_test, y_test_skb)

# Print results
print(f"Training Score: {lgb_train}")
print(f"Test Score: {lgb_test}")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.025965 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2553
[LightGBM] [Info] Number of data points in the train set: 767776, number of used features: 11
[LightGBM] [Info] Start training from score -0.966734
[LightGBM] [Info] Start training from score -2.506589
[LightGBM] [Info] Start training from score -0.619653
Training Score: 0.9900400637685992
Test Score: 0.989147933001641


# Creating Classification Reports

## Decision Tree

In [60]:
from sklearn.metrics import classification_report

# generate classification report
target_names = ['0', '1','2']
print(classification_report(y_test, y_pred_dt, target_names=target_names))

              precision    recall  f1-score   support

           0       0.94      0.96      0.95     73382
           1       0.78      0.71      0.75     15507
           2       1.00      1.00      1.00    103056

   micro avg       0.96      0.96      0.96    191945
   macro avg       0.91      0.89      0.90    191945
weighted avg       0.96      0.96      0.96    191945
 samples avg       0.96      0.96      0.96    191945



## Random Forest

In [61]:
from sklearn.metrics import classification_report

# generate classification report
target_names = ['0', '1','2']
print(classification_report(y_test, y_pred_rf, target_names=target_names))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99     73382
           1       0.96      0.94      0.95     15507
           2       1.00      1.00      1.00    103056

   micro avg       0.99      0.99      0.99    191945
   macro avg       0.98      0.98      0.98    191945
weighted avg       0.99      0.99      0.99    191945
 samples avg       0.99      0.99      0.99    191945



d:\python\anac\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Saving the Models

In [62]:
from joblib import dump,load
filename = 'random_forest.joblib'
# linear_svm, knn, decision tree, random forest
model = rf
dump(model, filename)

['random_forest.joblib']

In [63]:
ml_model=joblib.load('random_forest.joblib')

In [64]:
# for i in range(1,15):
#     output=ml_model.predict(X_train[i].reshape(1,-1))
#     if(output==0):
#         print("Data received is an outlier")
#     elif(output==1):
#         print("Data received is Malicious")
#     elif(output==2):
#         print("Data received is Benign")

In [65]:
for i in range(1, 15):
    output = ml_model.predict(X_train[i].reshape(1, -1)).flatten()[0]
    
    # Check if the output is an array or scalar
    if isinstance(output, np.ndarray):
        output = output[0]  # Extract scalar from array
        
    # Now output is guaranteed to be a scalar
    if output == 0:
        print("Data received is an outlier")
    elif output == 1:
        print("Data received is Malicious")
    elif output == 2:
        print("Data received is Benign")


Data received is an outlier
Data received is an outlier
Data received is an outlier
Data received is Malicious
Data received is Malicious
Data received is an outlier
Data received is Malicious
Data received is an outlier
Data received is an outlier
Data received is an outlier
Data received is an outlier
Data received is Malicious
Data received is Malicious
Data received is an outlier


SVC

In [ ]:
from sklearn.svm import SVC
model = SVC(kernel="rbf", random_state=42)
model.fit(X_train, y_train_skb)
print("Test Accuracy:", model.score(X_test, y_test_skb))


Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest
model = IsolationForest(contamination=0.05, random_state=42)
model.fit(X_train)
predictions = model.predict(X_test)  # -1 (outliers), 1 (normal)

Autoencoders (Deep Learning for Anomaly Detection)

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense

input_dim = X_train.shape[1]
input_layer = Input(shape=(input_dim,))
encoded = Dense(64, activation="relu")(input_layer)
encoded = Dense(32, activation="relu")(encoded)
decoded = Dense(64, activation="relu")(encoded)
decoded = Dense(input_dim, activation="sigmoid")(decoded)

autoencoder = Model(input_layer, decoded)
autoencoder.compile(optimizer='adam', loss='mse')

autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_test, X_test))
